# This Example shows how to classify EuroSAT satellite images using the deep learning model DeepSAT-V2.

### EuroSAT satellite dataset contains images from 10 different classes:

<ul>
<li>Annual Crop</li>
<li>Forest</li>
<li>Herbaceous Vegetation</li>
<li>Highway</li>
<li>Industrial</li>
<li>Pasture</li>
<li>Permanent Crop</li>
<li>Residential</li>
<li>River</li>
<li>SeaLake</li>
</ul>

### Import Modules

In [1]:
import sys
import os
import numpy as np
import matplotlib.pyplot as plt
import time
import torch
import torch.nn as nn
import torchvision.transforms as transforms

from geotorchai.models.raster import SatCNN
from geotorchai.preprocessing.torch_df import RasterClassificationDf
from geotorchai.preprocessing import SparkRegistration, load_geotiff_image_as_binary_data
from geotorchai.preprocessing.raster import RasterProcessing as rp
from geotorchai.utility import TorchAdapter

# Import Apache Sedona
from sedona.spark import *

## Import PySpark
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import col, udf, expr, array, concat


## Import distributed modules
from torch.utils.data import DistributedSampler, DataLoader
import torch.distributed as dist
from torch.nn.parallel import DistributedDataParallel as DDP
from pyspark.ml.torch.distributor import TorchDistributor
from petastorm import TransformSpec

## Define spark session and Register with GeoTorchAI

In [2]:
config = SedonaContext.builder().master(MASTER_URL).config('spark.jars.packages',
           'org.apache.sedona:sedona-spark-shaded-3.4_2.12:1.4.1,'
           'org.datasyslab:geotools-wrapper:1.4.0-28.2').getOrCreate()

spark = SedonaContext.create(config)
sc = spark.sparkContext

23/07/20 14:14:57 WARN Utils: Your hostname, Kanchans-Laptop.local resolves to a loopback address: 127.0.0.1; using 192.168.1.4 instead (on interface en0)
23/07/20 14:14:57 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Ivy Default Cache set to: /Users/kanchan/.ivy2/cache
The jars for the packages stored in: /Users/kanchan/.ivy2/jars
org.apache.sedona#sedona-spark-shaded-3.0_2.12 added as a dependency
org.datasyslab#geotools-wrapper added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-e96c1194-cfb6-4a8b-bc8b-ab13ead57951;1.0
	confs: [default]


:: loading settings :: url = jar:file:/Users/kanchan/program_files/spark-3.4.1-bin-hadoop3/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


	found org.apache.sedona#sedona-spark-shaded-3.0_2.12;1.4.1 in central
	found org.datasyslab#geotools-wrapper;1.4.0-28.2 in central
:: resolution report :: resolve 84ms :: artifacts dl 5ms
	:: modules in use:
	org.apache.sedona#sedona-spark-shaded-3.0_2.12;1.4.1 from central in [default]
	org.datasyslab#geotools-wrapper;1.4.0-28.2 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   2   |   0   |   0   |   0   ||   2   |   0   |
	---------------------------------------------------------------------
:: retrieving :: org.apache.spark#spark-submit-parent-e96c1194-cfb6-4a8b-bc8b-ab13ead57951
	confs: [default]
	0 artifacts copied, 2 already retrieved (0kB/3ms)
23/07/20 14:14:57 WARN NativeCodeLoader: Unable to load

### Register SparkSession with GeoTorchAI

In [3]:
SparkRegistration.set_spark_session(spark)

## Load Raster Data to Sedona

In [5]:
raster_df = load_geotiff_image_as_binary_data("data/eurosat_total/*.tif")
raster_df.show(5)

+--------------------+--------------------+------+--------------------+
|                path|    modificationTime|length|             content|
+--------------------+--------------------+------+--------------------+
|file:/Users/kanch...|2023-04-28 09:36:...|107244|[49 49 2A 00 08 0...|
|file:/Users/kanch...|2023-04-28 09:36:...|107244|[49 49 2A 00 08 0...|
|file:/Users/kanch...|2023-04-28 09:36:...|107244|[49 49 2A 00 08 0...|
|file:/Users/kanch...|2023-04-28 09:36:...|107244|[49 49 2A 00 08 0...|
|file:/Users/kanch...|2023-04-28 09:36:...|107244|[49 49 2A 00 08 0...|
+--------------------+--------------------+------+--------------------+
only showing top 5 rows



In [ ]:
raster_df = rp.get_array_from_binary_raster(raster_df, 13, "content", col_new_array_data="array_data")
raster_df.show(5)

In [12]:
raster_df.printSchema()

root
 |-- path: string (nullable = true)
 |-- modificationTime: timestamp (nullable = true)
 |-- length: long (nullable = true)
 |-- content: binary (nullable = true)
 |-- raster_data: raster (nullable = true)
 |-- array_data: array (nullable = true)
 |    |-- element: double (containsNull = true)



In [13]:
def get_name(file_path):
    return file_path.split("/")[-1].split("_")[0]

get_name_udf = udf(lambda x: get_name(x), StringType())

In [14]:
raster_df = raster_df.withColumn("category", get_name_udf(raster_df["path"]))
raster_df.show(5)

+--------------------+--------------------+------+--------------------+--------------------+--------------------+----------+
|                path|    modificationTime|length|             content|         raster_data|          array_data|  category|
+--------------------+--------------------+------+--------------------+--------------------+--------------------+----------+
|file:/Users/kanch...|2023-04-28 09:36:...|107244|[49 49 2A 00 08 0...|GridCoverage2D["g...|[1420.0, 1420.0, ...|AnnualCrop|
|file:/Users/kanch...|2023-04-28 09:36:...|107244|[49 49 2A 00 08 0...|GridCoverage2D["g...|[1133.0, 1133.0, ...|AnnualCrop|
|file:/Users/kanch...|2023-04-28 09:36:...|107244|[49 49 2A 00 08 0...|GridCoverage2D["g...|[1773.0, 1773.0, ...|AnnualCrop|
|file:/Users/kanch...|2023-04-28 09:36:...|107244|[49 49 2A 00 08 0...|GridCoverage2D["g...|[1388.0, 1388.0, ...|AnnualCrop|
|file:/Users/kanch...|2023-04-28 09:36:...|107244|[49 49 2A 00 08 0...|GridCoverage2D["g...|[1650.0, 1650.0, ...|AnnualCrop|


## Visualize the Bands of an Image

In [ ]:
img_index = 5
rp.visualize_all_bands(raster_df, "array_data", img_index, no_bands=13, height=64, width=64, axis_rows=2, axis_cols=7)

In [ ]:
objRasterClassify = RasterClassificationDf(raster_df, "array_data", "category", 64, 64, 13)
formatted_df = objRasterClassify.get_formatted_df()
transform_spec = objRasterClassify.get_transform_spec()

In [21]:
formatted_df.printSchema()

root
 |-- img_data: array (nullable = true)
 |    |-- element: double (containsNull = true)
 |-- label: integer (nullable = true)



In [22]:
from petastorm.spark import SparkDatasetConverter, make_spark_converter
spark.conf.set(SparkDatasetConverter.PARENT_CACHE_DIR_URL_CONF, "file:///tmp/petastorm")

/Users/kanchan/.pyenv/versions/3.11.0/lib/python3.11/site-packages/petastorm/spark/spark_dataset_converter.py:28: FutureWarning: pyarrow.LocalFileSystem is deprecated as of 2.0.0, please use pyarrow.fs.LocalFileSystem instead.
  from pyarrow import LocalFileSystem


In [24]:
converter_train = make_spark_converter(formatted_df)
print(len(converter_train))

/Users/kanchan/.pyenv/versions/3.11.0/lib/python3.11/site-packages/petastorm/fs_utils.py:88: FutureWarning: pyarrow.localfs is deprecated as of 2.0.0, please use pyarrow.fs.LocalFileSystem instead.
  self._filesystem = pyarrow.localfs
Converting floating-point columns to float32
The median size 2118207 B (< 50 MB) of the parquet files is too small. Total size: 20620744 B. Increase the median file size by calling df.repartition(n) or df.coalesce(n), which might help improve the performance. Parquet files: file:///tmp/petastorm/20230720141644-appid-local-1689887698493-cc88d6ce-9f17-47fb-b38e-db259aff486a/part-00002-6ef584f9-e0b5-4c2e-b1fb-3066f656407e-c000.parquet, ...


In [57]:
learning_rate = 0.0002
batch_size = 8
epoch_nums = 10

### Train Model
Accuracy will be very low since training is performed on a small dataset for only 2 epochs

In [60]:
def train_one_epoch(model, train_dataloader_iter, steps_per_epoch, optimizer, loss_fn, device):
    model.train()
    for step in range(steps_per_epoch):
        batch_data = next(train_dataloader_iter)
        inputs = batch_data['image_data']
        labels = batch_data['label']
        inputs = inputs.to(device)
        labels = labels.to(device)
            
        with torch.set_grad_enabled(True):
            optimizer.zero_grad()
                
            # Forward pass
            outputs = model(inputs)
            loss = loss_fn(outputs, labels)
                
            # Backward and optimize
            loss.backward()
            optimizer.step()
    return loss.item()

In [ ]:
def train_model(model, device):
    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    loss_fn.to(device)
    with converter_train.make_torch_dataloader(transform_spec=transform_spec, batch_size=batch_size) as train_dataloader:
        steps_per_epoch = len(converter_train) // batch_size
        train_dataloader_iter = iter(train_dataloader)
        for e in range(epoch_nums):
            epoch_loss = train_one_epoch(model, train_dataloader_iter, steps_per_epoch, optimizer, loss_fn)
            print('Epoch [{}/{}], Training Loss: {:.4f}'.format(e + 1, epoch_nums, epoch_loss))

In [ ]:
def train_distributed(use_gpu):
    backend = "nccl" if use_gpu else "gloo"
    dist.init_process_group(backend)
    device = int(os.environ["LOCAL_RANK"]) if use_gpu  else "cpu"
    model = SatCNN(13, 64, 64, 10).to(device)
    model = DDP(model)
    
    output = train_model(model, device)
    #dist.cleanup()

## Start Distributed Training

In [ ]:
distributor = TorchDistributor(num_processes=6, local_mode=True, use_gpu=False)
distributor.run(train_distributed, False)